In [ ]:
#spotifyAPI
!pip install spotipy
!pip install pprintpp

In [ ]:
import json
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.preprocessing import MinMaxScaler

# Stuff I imported
from pprintpp import pprint as pp
import json
from time import time, sleep
from random import randint

In [ ]:
client_id = '5cb8537300574d549c5983a2417f8392' #insert your client id
client_secret = 'af60768b38894f4c97aa0bae655dc7d9' # insert your client secret id here

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
playlist_id='49muI8fxakclsOqyShRkoV' #insert your playlist id
results = sp.playlist(playlist_id)

In [201]:
#------------------- I have added this cell --------------------#
#
# To get data from all the pages

track_id = []
playlist_info = []
results = sp.playlist(playlist_id)

# Will return a dictionary containing info of 100 tracks
tracks = results['tracks']

# Append the initial 100 tracks to the list
playlist_info.append(tracks)

song_meta={'id':[],'album':[], 'name':[], 'artist':[],'explicit':[],'popularity':[]}

i = 0
while (tracks['next'] and i < 7):
    # Gives a dictionary of the next 100 tracks
    next_tracks = sp.next(tracks)
    #print(tracks)

    playlist_info.append(next_tracks)
    
    # Use sleep in your project. I have given a break at i=7 because of time constraints.
    #sleep(randint(8,15))

    i+=1

# Use JSON to store the metadata
playlist_details = json.dumps(playlist_info, indent=4)
with open("playlist_info.json", "w") as write_file:
    write_file.write(playlist_details)

In [208]:
#------------------- I have added this cell --------------------#

# I recommend that you use JSON for storing and referencing while scraping. It will save you (a) A lot of time (b) unwanted requests to the API 
# (and hence the server) - which will reduce the chances of your request being denied.
#
# When I tried using your code in the following cell, the server rejected requests as they were many in number. You are getting the metadata 
# in the previous cell, why not use it?
#
# Doing this is optional and you may choose to not follow it.
# But if you do, please keep in mind that the way you are accessing audio_features later in the code will change.
# Also note that the way you are merging the two dataframes towards the end might also change (check for duplicates).

song_meta={'id':[],'album':[], 'name':[], 
           'artist':[],'explicit':[],'popularity':[]}

with open("playlist_info.json", "r") as read_file:
    playlist_info = json.loads(read_file.read())

for playlist in playlist_info:
    for items in playlist['items']:
        # Album Name
        song_meta['album'].append(items['track']['album']['name'])

        # Song ID
        song_meta['id'].append(items['track']['id'])
        
        # Song Name
        song_meta['name'].append(items['track']['name'])

        # Artists Name
        artists_list = []
        artists = items['track']['artists']
        for artist in artists:
            artists_list.append(artist['name'])
        song_meta['artist'].append(artists_list)

        # Explicit
        song_meta['explicit'].append(items['track']['explicit'])

        # Popularity
        song_meta['popularity'].append(items['track']['popularity'])

#pp(song_meta)
print("Album Name :", len(song_meta['album']))
print("Song ID :", len(song_meta['id']))
print("Song Name :", len(song_meta['name']))
print("Artists Name :", len(song_meta['artist']))
print("Explicit :", len(song_meta['explicit']))
print("Popularity :", len(song_meta['popularity']))
song_meta_df=pd.DataFrame.from_dict(song_meta)
song_meta_df.info()

Album Name : 800
Song ID : 800
Song Name : 800
Artists Name : 800
Explicit : 800
Popularity : 800
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          800 non-null    object
 1   album       800 non-null    object
 2   name        800 non-null    object
 3   artist      800 non-null    object
 4   explicit    800 non-null    bool  
 5   popularity  800 non-null    int64 
dtypes: bool(1), int64(1), object(4)
memory usage: 32.2+ KB


In [ ]:
# create a list of song ids
ids=[]

for item in results['tracks']['items']:
        track = item['track']['id']
        ids.append(track)
        
song_meta={'id':[],'album':[], 'name':[], 
           'artist':[],'explicit':[],'popularity':[]}

for song_id in ids:
    # get song's meta data
    meta = sp.track(song_id)
    
    # song id
    song_meta['id'].append(song_id)

    # album name
    album=meta['album']['name']
    song_meta['album']+=[album]

    # song name
    song=meta['name']
    song_meta['name']+=[song]
    
    # artists name
    s = ', '
    artist=s.join([singer_name['name'] for singer_name in meta['artists']])
    song_meta['artist']+=[artist]
    
    # explicit: lyrics could be considered offensive or unsuitable for children
    explicit=meta['explicit']
    song_meta['explicit'].append(explicit)
    
    # song popularity
    popularity=meta['popularity']
    song_meta['popularity'].append(popularity)

song_meta_df=pd.DataFrame.from_dict(song_meta)

In [204]:
#song_meta_df=pd.DataFrame.from_dict(song_meta)
#print(song_meta_df)

                        id  ... popularity
0   7hQJA50XrCWABAu5v6QZ4i  ...         74
1   7AgMX97ljLm9BzszyyXEYZ  ...         58
2   2rk0lCpO1vkKAQ6BC8bjUX  ...         60
3   7uR7WsWw2HTKDnz4Qw6wTH  ...         51
4   20XdEFyaUR9C7aDIdq2OAd  ...         63
..                     ...  ...        ...
89  0v8DQWCyaGcczI1SAT63Gc  ...          3
90  7DOrSrAyBadp2vAETJ290l  ...          3
91  3FBOfmWp97g5YIpBzOmItj  ...          3
92  13muSTfToQAkq9oSIoQKXH  ...          2
93  0tZ0fbxddRd1XixV7bt4df  ...          3

[94 rows x 6 columns]


In [209]:
#------------------- I have edited this cell --------------------#

# check the song feature
features = []
i = 0
for id in song_meta['id']:
    audio_features = sp.audio_features(id)[0]

    if audio_features!=None:
        features.append(audio_features)
    #print(features)
    # Run it entirely for your project! Do not break.
    if (i==500):
        break
    i+=1

#pp(features)
features_df=pd.DataFrame(features)

In [210]:
features_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.563,0.865,5,-5.277,1,0.1600,0.0472,0.000191,0.7700,0.6010,156.271,audio_features,7hQJA50XrCWABAu5v6QZ4i,spotify:track:7hQJA50XrCWABAu5v6QZ4i,https://api.spotify.com/v1/tracks/7hQJA50XrCWA...,https://api.spotify.com/v1/audio-analysis/7hQJ...,209413,4
1,0.822,0.479,11,-15.962,1,0.2710,0.7190,0.000044,0.1240,0.7200,92.461,audio_features,7AgMX97ljLm9BzszyyXEYZ,spotify:track:7AgMX97ljLm9BzszyyXEYZ,https://api.spotify.com/v1/tracks/7AgMX97ljLm9...,https://api.spotify.com/v1/audio-analysis/7AgM...,250240,4
2,0.636,0.610,8,-9.665,1,0.1190,0.2500,0.000121,0.1130,0.2960,125.227,audio_features,2rk0lCpO1vkKAQ6BC8bjUX,spotify:track:2rk0lCpO1vkKAQ6BC8bjUX,https://api.spotify.com/v1/tracks/2rk0lCpO1vkK...,https://api.spotify.com/v1/audio-analysis/2rk0...,260160,3
3,0.761,0.155,8,-21.639,1,0.0617,0.6700,0.522000,0.0820,0.7070,114.126,audio_features,7uR7WsWw2HTKDnz4Qw6wTH,spotify:track:7uR7WsWw2HTKDnz4Qw6wTH,https://api.spotify.com/v1/tracks/7uR7WsWw2HTK...,https://api.spotify.com/v1/audio-analysis/7uR7...,153590,4
4,0.677,0.540,7,-14.439,1,0.0627,0.0455,0.000002,0.8450,0.4690,95.116,audio_features,20XdEFyaUR9C7aDIdq2OAd,spotify:track:20XdEFyaUR9C7aDIdq2OAd,https://api.spotify.com/v1/tracks/20XdEFyaUR9C...,https://api.spotify.com/v1/audio-analysis/20Xd...,270973,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,0.429,0.247,9,-15.478,1,0.0375,0.9280,0.944000,0.0906,0.0575,136.498,audio_features,1dx1aLzW5bGvk9bg03aHYa,spotify:track:1dx1aLzW5bGvk9bg03aHYa,https://api.spotify.com/v1/tracks/1dx1aLzW5bGv...,https://api.spotify.com/v1/audio-analysis/1dx1...,125062,4
493,0.418,0.590,7,-14.370,0,0.0443,0.9550,0.871000,0.1370,0.0536,96.682,audio_features,0lPAxxVb9BXDi82EvCFRa1,spotify:track:0lPAxxVb9BXDi82EvCFRa1,https://api.spotify.com/v1/tracks/0lPAxxVb9BXD...,https://api.spotify.com/v1/audio-analysis/0lPA...,145840,4
494,0.691,0.179,7,-15.682,1,0.0373,0.8420,0.931000,0.0943,0.2220,108.573,audio_features,6zeTz2j5A9G6gEKi34ks05,spotify:track:6zeTz2j5A9G6gEKi34ks05,https://api.spotify.com/v1/tracks/6zeTz2j5A9G6...,https://api.spotify.com/v1/audio-analysis/6zeT...,205872,4
495,0.713,0.415,10,-11.179,0,0.0389,0.0274,0.539000,0.1060,0.6420,89.981,audio_features,34KMYyUvfpXboR9wLwrocX,spotify:track:34KMYyUvfpXboR9wLwrocX,https://api.spotify.com/v1/tracks/34KMYyUvfpXb...,https://api.spotify.com/v1/audio-analysis/34KM...,152000,4


In [ ]:
#------- Might need a little tweaking ----------#

# convert milliseconds to mins
# duration_ms: The duration of the track in milliseconds.
# 1 minute = 60 seconds = 60 × 1000 milliseconds = 60,000 ms
features_df['duration_ms']=features_df['duration_ms']/60000

# combine two dataframe
final_df=song_meta_df.merge(features_df)

In [ ]:
final_df

,id,album,name,artist,explicit,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,4bp58Gcr7KwVwZOFR6sFcj,Formal Jazz Session,Finest JazzHop,"[Jazz Fruits Music, Formal Chicken, Lofi Fruit...",False,54,0.901,0.481,11,-6.860,0,0.0401,0.3500,0.201,0.0732,0.363,95.006,audio_features,spotify:track:4bp58Gcr7KwVwZOFR6sFcj,https://api.spotify.com/v1/tracks/4bp58Gcr7KwV...,https://api.spotify.com/v1/audio-analysis/4bp5...,2.109617,4
1,4bp58Gcr7KwVwZOFR6sFcj,Formal Jazz Session,Finest JazzHop,"[Jazz Fruits Music, Formal Chicken, Lofi Fruit...",False,54,0.901,0.481,11,-6.860,0,0.0401,0.3500,0.201,0.0732,0.363,95.006,audio_features,spotify:track:4bp58Gcr7KwVwZOFR6sFcj,https://api.spotify.com/v1/tracks/4bp58Gcr7KwV...,https://api.spotify.com/v1/audio-analysis/4bp5...,2.109617,4
2,4bp58Gcr7KwVwZOFR6sFcj,Formal Jazz Session,Finest JazzHop,"[Jazz Fruits Music, Formal Chicken, Lofi Fruit...",False,54,0.901,0.481,11,-6.860,0,0.0401,0.3500,0.201,0.0732,0.363,95.006,audio_features,spotify:track:4bp58Gcr7KwVwZOFR6sFcj,https://api.spotify.com/v1/tracks/4bp58Gcr7KwV...,https://api.spotify.com/v1/audio-analysis/4bp5...,2.109617,4
3,4bp58Gcr7KwVwZOFR6sFcj,Formal Jazz Session,Finest JazzHop,"[Jazz Fruits Music, Formal Chicken, Lofi Fruit...",False,54,0.901,0.481,11,-6.860,0,0.0401,0.3500,0.201,0.0732,0.363,95.006,audio_features,spotify:track:4bp58Gcr7KwVwZOFR6sFcj,https://api.spotify.com/v1/tracks/4bp58Gcr7KwV...,https://api.spotify.com/v1/audio-analysis/4bp5...,2.109617,4
4,4bp58Gcr7KwVwZOFR6sFcj,Formal Jazz Session,Finest JazzHop,"[Jazz Fruits Music, Formal Chicken, Lofi Fruit...",False,54,0.901,0.481,11,-6.860,0,0.0401,0.3500,0.201,0.0732,0.363,95.006,audio_features,spotify:track:4bp58Gcr7KwVwZOFR6sFcj,https://api.spotify.com/v1/tracks/4bp58Gcr7KwV...,https://api.spotify.com/v1/audio-analysis/4bp5...,2.109617,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3467,34KMYyUvfpXboR9wLwrocX,Mental Acupuncture,Mental Acupuncture,[Jazzinuf],False,54,0.713,0.415,10,-11.179,0,0.0389,0.0274,0.539,0.1060,0.642,89.981,audio_features,spotify:track:34KMYyUvfpXboR9wLwrocX,https://api.spotify.com/v1/tracks/34KMYyUvfpXb...,https://api.spotify.com/v1/audio-analysis/34KM...,2.533333,4
3468,34KMYyUvfpXboR9wLwrocX,Mental Acupuncture,Mental Acupuncture,[Jazzinuf],False,54,0.713,0.415,10,-11.179,0,0.0389,0.0274,0.539,0.1060,0.642,89.981,audio_features,spotify:track:34KMYyUvfpXboR9wLwrocX,https://api.spotify.com/v1/tracks/34KMYyUvfpXb...,https://api.spotify.com/v1/audio-analysis/34KM...,2.533333,4
3469,34KMYyUvfpXboR9wLwrocX,Mental Acupuncture,Mental Acupuncture,[Jazzinuf],False,54,0.713,0.415,10,-11.179,0,0.0389,0.0274,0.539,0.1060,0.642,89.981,audio_features,spotify:track:34KMYyUvfpXboR9wLwrocX,https://api.spotify.com/v1/tracks/34KMYyUvfpXb...,https://api.spotify.com/v1/audio-analysis/34KM...,2.533333,4
3470,34KMYyUvfpXboR9wLwrocX,Mental Acupuncture,Mental Acupuncture,[Jazzinuf],False,54,0.713,0.415,10,-11.179,0,0.0389,0.0274,0.539,0.1060,0.642,89.981,audio_features,spotify:track:34KMYyUvfpXboR9wLwrocX,https://api.spotify.com/v1/tracks/34KMYyUvfpXb...,https://api.spotify.com/v1/audio-analysis/34KM...,2.533333,4


In [ ]:
music_feature=features_df[['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']]

In [ ]:
music_feature.describe()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
count,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.401046,0.229431,-20.72718,0.081785,0.732171,0.631291,0.164759,0.262755,96.804070,2.295431
std,0.259965,0.251687,9.45611,0.091963,0.333899,0.381747,0.144557,0.259891,31.974121,0.891903
min,0.076700,0.000849,-40.97400,0.031800,0.002780,0.000000,0.040800,0.029700,51.213000,1.066667
25%,0.179750,0.038400,-26.87875,0.041875,0.605750,0.277250,0.096700,0.039675,75.134500,1.639108
50%,0.282500,0.101200,-22.36300,0.047800,0.915000,0.849000,0.110500,0.137000,85.979000,1.923900
75%,0.669500,0.410500,-12.53225,0.075400,0.979000,0.932500,0.147750,0.405500,114.173000,2.816542
max,0.902000,0.895000,-5.27700,0.650000,0.996000,0.994000,0.845000,0.907000,205.348000,4.955333


In [ ]:
min_max_scaler = MinMaxScaler()
music_feature.loc[:]=min_max_scaler.fit_transform(music_feature.loc[:])

C:\Python38\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Python38\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [ ]:
music_feature.describe()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.393004,0.255641,0.567185,0.080856,0.734370,0.635102,0.154140,0.265650,0.295787,0.315986
std,0.314994,0.281482,0.264899,0.148759,0.336179,0.384051,0.179753,0.296239,0.207442,0.229360
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.124864,0.041996,0.394858,0.016297,0.607086,0.278924,0.069510,0.011370,0.155198,0.147208
50%,0.249364,0.112230,0.521360,0.025882,0.918447,0.854125,0.086670,0.122307,0.225556,0.220444
75%,0.718284,0.458145,0.796755,0.070527,0.982884,0.938129,0.132989,0.428360,0.408473,0.449994
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
uri_array = []

def showTracks(tracks, track_id):
    for i, item in enumerate(tracks['items']):
        track = item['track']
        uri_array.append(track['id'])